<h2><center>ANALYSIS ZOMATO DATASET</center></h2>

### ABOUT DATASET

#### 
   - <b> url </b>:  contains the url of the restaurant in the zomato website

   - <b> address </b> :  contains the address of the restaurant in Bengaluru

   - <b> name </b>:  contains the name of the restaurant

   - <b> online_order </b> :  whether online ordering is available in the restaurant or not

   - <b> book_table</b> :  table book option available or not

   - <b> rate </b> :  contains the overall rating of the restaurant out of 5

   - <b> votes </b> :  contains total number of rating for the restaurant as of the above mentioned date

   - <b> phone </b> :  contains the phone number of the restaurant

   - <b> location </b> :  contains the neighborhood in which the restaurant is located

   - <b> rest_type </b> :  restaurant type

   - <b> dish_liked </b> :  dishes people liked in the restaurant

   - <b> cuisines </b> :  food styles, separated by comma

   - <b> approx_cost(for two people) </b> :  contains the approximate cost for meal for two people

   - <b> reviews_list </b> :  list of tuples containing reviews for the restaurant, each tuple consists of two values, rating and review by the customer
 
   - <b> menu_item </b> :  contains list of menus available in the restaurant

   - <b> listed_in(type) </b> :  type of meal

   - <b> listed_in(city) </b> :  contains the neighborhood in which the restaurant is listed

####  

#### IMPORTING PACKAGES

##### FOR ACCESSING CLIENT

In [1]:
# !pip install pymongo
# !pip install plotly
# !pip install kaggle


In [2]:
# !ls

In [3]:
# !unzip zomato.csv.zip


In [4]:
import pymongo
from pymongo import MongoClient 
    


In [5]:


# Connect with the portnumber and host  
client = MongoClient("mongodb://localhost:27017/")  


#### READING AND ANALYSING THE DATASET

##### ACCESSING DATABASE

In [6]:
dblist=client.list_database_names()
dblist

['admin', 'config', 'local']

In [7]:
# Access database  
mydb = client['admin'] 
collection = mydb['zomo']

### RECOMMANDATION SYSTEM

In [8]:
# !pip install nltk


In [9]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
zomato_real= collection.aggregate([{"$project": {"name":"$name" ,"approx_cost(for two people)":"$approx_cost(for two people)",
                                            "rate":"$rate","reviews_list":"$reviews_list","cuisines":"$cuisines"}}])


In [11]:
zomato_real=pd.DataFrame(zomato_real)

In [12]:
zomato_real.head()

,_id,name,approx_cost(for two people),rate,reviews_list,cuisines
0,671740a9ca8d0b834008d7b2,Jalsa,800,4.1/5,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese"
1,671740a9ca8d0b834008d7b3,Spice Elephant,800,4.1/5,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai"
2,671740a9ca8d0b834008d7b4,San Churro Cafe,800,3.8/5,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian"
3,671740a9ca8d0b834008d7b5,Addhuri Udupi Bhojana,300,3.7/5,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian"
4,671740a9ca8d0b834008d7b6,Grand Village,600,3.8/5,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani"


In [13]:
df = zomato_real

In [14]:
name = list(df.name.unique())

In [15]:
df.name.nunique()

8792

In [16]:
df.columns

Index(['_id', 'name', 'approx_cost(for two people)', 'rate', 'reviews_list',
       'cuisines'],
      dtype='object')

In [17]:
df

,_id,name,approx_cost(for two people),rate,reviews_list,cuisines
0,671740a9ca8d0b834008d7b2,Jalsa,800,4.1/5,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese"
1,671740a9ca8d0b834008d7b3,Spice Elephant,800,4.1/5,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai"
2,671740a9ca8d0b834008d7b4,San Churro Cafe,800,3.8/5,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian"
3,671740a9ca8d0b834008d7b5,Addhuri Udupi Bhojana,300,3.7/5,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian"
4,671740a9ca8d0b834008d7b6,Grand Village,600,3.8/5,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani"
...,...,...,...,...,...,...
51712,671740aaca8d0b834009a1b2,Best Brews - Four Points by Sheraton Bengaluru...,"1,500",3.6 /5,"[('Rated 5.0', ""RATED\n Food and service are ...",Continental
51713,671740aaca8d0b834009a1b3,Vinod Bar And Restaurant,600,None,[],Finger Food
51714,671740aaca8d0b834009a1b4,Plunge - Sheraton Grand Bengaluru Whitefield H...,"2,000",None,[],Finger Food
51715,671740aaca8d0b834009a1b5,Chime - Sheraton Grand Bengaluru Whitefield Ho...,"2,500",4.3 /5,"[('Rated 4.0', 'RATED\n Nice and friendly pla...",Finger Food


In [18]:
new_shop =df[df["rate"] == "NEW" ]

In [19]:
(new_shop.shape)[0]

2208

In [20]:
# Remove rows where 'column_name' contains the substring 'new'
df = df[~df['rate'].str.contains('NEW', case=False, na=False)]


In [21]:
# Remove rows where 'rate' contains 'new' or is None
df = df[~df['rate'].str.contains('new', case=False, na=False) & df['rate'].notna()]


In [22]:
# Remove rows where 'column_name' contains the substring 'new'
df = df[~df['rate'].str.contains('-', case=False, na=False)]


In [23]:
df

,_id,name,approx_cost(for two people),rate,reviews_list,cuisines
0,671740a9ca8d0b834008d7b2,Jalsa,800,4.1/5,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese"
1,671740a9ca8d0b834008d7b3,Spice Elephant,800,4.1/5,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai"
2,671740a9ca8d0b834008d7b4,San Churro Cafe,800,3.8/5,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian"
3,671740a9ca8d0b834008d7b5,Addhuri Udupi Bhojana,300,3.7/5,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian"
4,671740a9ca8d0b834008d7b6,Grand Village,600,3.8/5,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani"
...,...,...,...,...,...,...
51709,671740aaca8d0b834009a1af,The Farm House Bar n Grill,800,3.7 /5,"[('Rated 4.0', 'RATED\n Ambience- Big and spa...","North Indian, Continental"
51711,671740aaca8d0b834009a1b1,Bhagini,800,2.5 /5,"[('Rated 4.0', 'RATED\n A fine place to chill...","Andhra, South Indian, Chinese, North Indian"
51712,671740aaca8d0b834009a1b2,Best Brews - Four Points by Sheraton Bengaluru...,"1,500",3.6 /5,"[('Rated 5.0', ""RATED\n Food and service are ...",Continental
51715,671740aaca8d0b834009a1b5,Chime - Sheraton Grand Bengaluru Whitefield Ho...,"2,500",4.3 /5,"[('Rated 4.0', 'RATED\n Nice and friendly pla...",Finger Food


In [24]:
df["rate"].nunique()

62

In [25]:
rate =df["rate"].unique()

In [26]:
rate

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5', '4.5/5',
       '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5', '3.4 /5',
       '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5', '4.1 /5',
       '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5', '3.5 /5',
       '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5', '4.3 /5',
       '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5', '4.9 /5',
       '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5', '2.1 /5',
       '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [27]:
df[df["rate"]=='-'].shape

(0, 6)

In [28]:
df['rating'] = df['rate'].str.extract(r'(\d+\.\d+)')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [29]:
df.dtypes

_id                             object
name                            object
approx_cost(for two people)     object
rate                            object
reviews_list                    object
cuisines                        object
rating                         float64
dtype: object

### Removing the Cost column data that contains None and lest than 100

In [30]:
df =df[df["approx_cost(for two people)"].notna() ]

In [31]:
df["approx_cost(for two people)"].unique()

array([800, 300, 600, 700, 550, 500, 450, 650, 400, 900, 200, 750, 150,
       850, 100, '1,200', 350, 250, 950, '1,000', '1,500', '1,300', 199,
       '1,100', '1,600', 230, 130, '1,700', '1,350', '2,200', '1,400',
       '2,000', '1,800', '1,900', 180, 330, '2,500', '2,100', '3,000',
       '2,800', '3,400', 50, 40, '1,250', '3,500', '4,000', '2,400',
       '2,600', '1,450', 70, '3,200', 240, '6,000', '1,050', '2,300',
       '4,100', 120, '5,000', '3,700', '1,650', '2,700', '4,500', 80],
      dtype=object)

### Converting to numeric by removing comma from the cost 

In [32]:
df['Cost_For_two'] = df['approx_cost(for two people)'].replace(',', '', regex=True)  # Remove commas
df['Cost_For_two'] = pd.to_numeric(df['Cost_For_two'], errors='coerce')  # Convert to numeric


In [33]:
(df[df['Cost_For_two'] <200].shape)[0]

1998

In [34]:
df = df[df['Cost_For_two'] >100]

In [35]:
df["approx_cost(for two people)"].nunique()

58

In [36]:
cost =df["Cost_For_two"].unique()

In [37]:
cost

array([ 800,  300,  600,  700,  550,  500,  450,  650,  400,  900,  200,
        750,  150,  850, 1200,  350,  250,  950, 1000, 1500, 1300,  199,
       1100, 1600,  230,  130, 1700, 1350, 2200, 1400, 2000, 1800, 1900,
        180,  330, 2500, 2100, 3000, 2800, 3400, 1250, 3500, 4000, 2400,
       2600, 1450, 3200,  240, 6000, 1050, 2300, 4100,  120, 5000, 3700,
       1650, 2700, 4500])

In [38]:
df.columns

Index(['_id', 'name', 'approx_cost(for two people)', 'rate', 'reviews_list',
       'cuisines', 'rating', 'Cost_For_two'],
      dtype='object')

In [39]:
# Exclude specific columns
df = df.drop(columns=['approx_cost(for two people)', 'rate'])


In [40]:
df

,_id,name,reviews_list,cuisines,rating,Cost_For_two
0,671740a9ca8d0b834008d7b2,Jalsa,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese",4.1,800
1,671740a9ca8d0b834008d7b3,Spice Elephant,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai",4.1,800
2,671740a9ca8d0b834008d7b4,San Churro Cafe,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian",3.8,800
3,671740a9ca8d0b834008d7b5,Addhuri Udupi Bhojana,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian",3.7,300
4,671740a9ca8d0b834008d7b6,Grand Village,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani",3.8,600
...,...,...,...,...,...,...
51709,671740aaca8d0b834009a1af,The Farm House Bar n Grill,"[('Rated 4.0', 'RATED\n Ambience- Big and spa...","North Indian, Continental",3.7,800
51711,671740aaca8d0b834009a1b1,Bhagini,"[('Rated 4.0', 'RATED\n A fine place to chill...","Andhra, South Indian, Chinese, North Indian",2.5,800
51712,671740aaca8d0b834009a1b2,Best Brews - Four Points by Sheraton Bengaluru...,"[('Rated 5.0', ""RATED\n Food and service are ...",Continental,3.6,1500
51715,671740aaca8d0b834009a1b5,Chime - Sheraton Grand Bengaluru Whitefield Ho...,"[('Rated 4.0', 'RATED\n Nice and friendly pla...",Finger Food,4.3,2500


## Normalizing the Name , cuisine columns and creating Cuisine_Count


In [41]:

# Specify the columns you want to normalize
columns_to_normalize = ['name', 'cuisines']

# Convert specified columns to lowercase
for col in columns_to_normalize:
    df[col] = df[col].str.lower()




In [42]:
# Handle None values by applying a condition
df['Cuisine_Count'] = df['cuisines'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)

df

,_id,name,reviews_list,cuisines,rating,Cost_For_two,Cuisine_Count
0,671740a9ca8d0b834008d7b2,jalsa,"[('Rated 4.0', 'RATED\n A beautiful place to ...","north indian, mughlai, chinese",4.1,800,3
1,671740a9ca8d0b834008d7b3,spice elephant,"[('Rated 4.0', 'RATED\n Had been here for din...","chinese, north indian, thai",4.1,800,3
2,671740a9ca8d0b834008d7b4,san churro cafe,"[('Rated 3.0', ""RATED\n Ambience is not that ...","cafe, mexican, italian",3.8,800,3
3,671740a9ca8d0b834008d7b5,addhuri udupi bhojana,"[('Rated 4.0', ""RATED\n Great food and proper...","south indian, north indian",3.7,300,2
4,671740a9ca8d0b834008d7b6,grand village,"[('Rated 4.0', 'RATED\n Very good restaurant ...","north indian, rajasthani",3.8,600,2
...,...,...,...,...,...,...,...
51709,671740aaca8d0b834009a1af,the farm house bar n grill,"[('Rated 4.0', 'RATED\n Ambience- Big and spa...","north indian, continental",3.7,800,2
51711,671740aaca8d0b834009a1b1,bhagini,"[('Rated 4.0', 'RATED\n A fine place to chill...","andhra, south indian, chinese, north indian",2.5,800,4
51712,671740aaca8d0b834009a1b2,best brews - four points by sheraton bengaluru...,"[('Rated 5.0', ""RATED\n Food and service are ...",continental,3.6,1500,1
51715,671740aaca8d0b834009a1b5,chime - sheraton grand bengaluru whitefield ho...,"[('Rated 4.0', 'RATED\n Nice and friendly pla...",finger food,4.3,2500,1


In [43]:
# Find the row with the maximum number of cuisines
max_cuisine_row = df.loc[df['Cuisine_Count'].idxmax()]
max_cuisine_row

_id                                       671740a9ca8d0b834008d7e9
name                                                     freshmenu
reviews_list     [('Rated 5.0', 'RATED\n  What: continental foo...
cuisines         healthy food, chinese, biryani, north indian, ...
rating                                                         3.9
Cost_For_two                                                   450
Cuisine_Count                                                    8
Name: 55, dtype: object

In [44]:
# Find the row with the maximum number of cuisines
min_cuisine_row = df.loc[df['Cuisine_Count'].idxmin()]
min_cuisine_row

_id                                       671740a9ca8d0b834008f299
name                                                  noodle oodle
reviews_list     [('Rated 4.0', 'RATED\n  Lil oily else good'),...
cuisines                                                      None
rating                                                         3.6
Cost_For_two                                                   400
Cuisine_Count                                                    0
Name: 6887, dtype: object

In [61]:
# Six hotels with 0 cuisines
(df[df['Cuisine_Count'] ==0].shape)[0]

6

In [62]:
df[df['Cuisine_Count'] ==0]

,_id,name,reviews_list,cuisines,rating,Cost_For_two,Cuisine_Count
6887,671740a9ca8d0b834008f299,noodle oodle,"[('Rated 4.0', 'RATED\n Lil oily else good'),...",None,3.6,400,0
24725,671740aaca8d0b8340093847,swagatham rayalaseema ruchulu,"[('Rated 1.0', ""RATED\n Mediocre!\n\nWe went ...",None,3.3,600,0
26186,671740aaca8d0b8340093dfc,swagatham rayalaseema ruchulu,"[('Rated 1.0', ""RATED\n Mediocre!\n\nWe went ...",None,3.3,600,0
40625,671740aaca8d0b8340097663,noodle oodle,"[('Rated 4.0', 'RATED\n Lil oily else good'),...",None,3.7,400,0
50355,671740aaca8d0b8340099c65,noodle oodle,"[('Rated 4.0', 'RATED\n Lil oily else good'),...",None,3.7,400,0
50439,671740aaca8d0b8340099cb9,taste of chettinad,"[('Rated 1.0', 'RATED\n The biryani was not e...",None,3.2,400,0


In [63]:
# Six hotels with 0 cuisines
(df[df['Cuisine_Count'] ==1].shape)[0]

8455

In [64]:
df[df['Cuisine_Count'] ==1]

,_id,name,reviews_list,cuisines,rating,Cost_For_two,Cuisine_Count
5,671740a9ca8d0b834008d7b7,timepass dinner,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",north indian,3.8,600,1
10,671740a9ca8d0b834008d7bc,cafãâãâãâãâãâãâãâãâ© down the a...,"[('Rated 4.0', 'RATED\n We ended up here on a...",cafe,4.1,500,1
15,671740a9ca8d0b834008d7c1,cafe vivacity,"[('Rated 2.0', 'RATED\n Not so good place as ...",cafe,3.8,650,1
24,671740a9ca8d0b834008d7ca,hide out cafe,"[('Rated 4.0', 'RATED\n The food was quite go...",cafe,3.7,300,1
33,671740a9ca8d0b834008d7d3,ovenstory pizza,"[('Rated 4.0', 'RATED\n Stumbled upon this on...",pizza,3.9,750,1
...,...,...,...,...,...,...,...
51681,671740aaca8d0b834009a193,chime - sheraton grand bengaluru whitefield ho...,"[('Rated 4.0', 'RATED\n Nice and friendly pla...",finger food,4.3,2500,1
51683,671740aaca8d0b834009a195,alt,"[('Rated 3.0', ""RATED\n It's a skydeck which ...",finger food,4.1,1900,1
51707,671740aaca8d0b834009a1ad,m bar - bengaluru marriott hotel whitefield,"[('Rated 4.0', 'RATED\n Went there post dinne...",finger food,3.9,2000,1
51712,671740aaca8d0b834009a1b2,best brews - four points by sheraton bengaluru...,"[('Rated 5.0', ""RATED\n Food and service are ...",continental,3.6,1500,1


In [1]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [2]:
import folium

# Coordinates for Bangalore (latitude and longitude)
bangalore_coords = [12.9716, 77.5946]

# Create a map centered on Bangalore
bangalore_map = folium.Map(location=bangalore_coords, zoom_start=12)

# Example: Add a marker for a specific location (like a street or area)
folium.Marker([12.934533, 77.626579], popup="Koramangala").add_to(bangalore_map)

# Example: Draw a polygon for a particular area (let's assume it's a custom area)
folium.Polygon(locations=[
    [12.934533, 77.626579],
    [12.940707, 77.622857],
    [12.947403, 77.634372],
    [12.943155, 77.640607]
], color="blue", weight=2.5, fill=True, fill_color="blue", fill_opacity=0.5).add_to(bangalore_map)

# Save the map to an HTML file
bangalore_map.save("bangalore_map.html")


In [46]:
# zomato_real.to_csv('zomato_recommendation_data.csv',index=False)

Now the next step is data cleaning and feature engineering for this step we need to do a lot of stuff with the data such as:

    Deleting Unnecessary Columns
    Removing the Duplicates
    Remove the NaN values from the dataset
    Changing the column names
    Data Transformations
    Data Cleaning
    Adjust the column names Now, let’s perform all the above steps in our data

In [47]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['_id'],axis=1) #Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"
# zomato=zomato_real[['name','approx_cost(for two people)','rate','reviews_list','cuisines']]
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)

In [48]:
zomato

,name,approx_cost(for two people),rate,reviews_list,cuisines
0,Jalsa,800,4.1/5,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese"
1,Spice Elephant,800,4.1/5,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai"
2,San Churro Cafe,800,3.8/5,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian"
3,Addhuri Udupi Bhojana,300,3.7/5,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian"
4,Grand Village,600,3.8/5,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani"
...,...,...,...,...,...
51699,Harry's Bar + Cafe,"1,200",4.1 /5,"[('Rated 4.0', 'RATED\n The first thing that ...","Asian, American, Mexican"
51700,BAR BAR,"1,500",4.1 /5,"[('Rated 4.0', ""RATED\n In an increasingly ov...","Continental, North Indian, Italian"
51701,The Beer Cafe,"1,400",4.1 /5,"[('Rated 2.0', ""RATED\n We went out to this p...","Finger Food, North Indian, Chinese, Italian"
51702,Pablo's Gastrobar,"1,450",4.0 /5,"[('Rated 4.0', 'RATED\n A good place to dine....","Finger Food, American, Asian, Italian"


In [49]:
# Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)': 'cost'})

# Some Transformations
zomato['cost'] = zomato['cost'].astype(str)  # Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',', '.'))  # Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)

# Removing '/5' from Rates
zomato = zomato.loc[zomato.rate != 'NEW']
zomato = zomato.loc[zomato.rate != '-'].reset_index(drop=True)

# Update the remove_slash lambda function
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x
zomato['rate'] = zomato['rate'].apply(remove_slash).str.strip().astype(float)

# Adjust the column names
zomato['name'] = zomato['name'].apply(lambda x: x.title())

# Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0.0  # Initialize with float

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

In [50]:
zomato.head()

,name,cost,rate,reviews_list,cuisines,Mean Rating
0,Jalsa,800.0,4.1,"[('Rated 4.0', 'RATED\n A beautiful place to ...","North Indian, Mughlai, Chinese",4.00
1,Spice Elephant,800.0,4.1,"[('Rated 4.0', 'RATED\n Had been here for din...","Chinese, North Indian, Thai",3.97
2,San Churro Cafe,800.0,3.8,"[('Rated 3.0', ""RATED\n Ambience is not that ...","Cafe, Mexican, Italian",3.58
3,Addhuri Udupi Bhojana,300.0,3.7,"[('Rated 4.0', ""RATED\n Great food and proper...","South Indian, North Indian",3.45
4,Grand Village,600.0,3.8,"[('Rated 4.0', 'RATED\n Very good restaurant ...","North Indian, Rajasthani",3.58


Now the next step is to perform some text preprocessing steps which include:

    Lower casing
    Removal of Punctuations
    Removal of Stopwords
    Removal of URLs
    Spelling correction

In [51]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/srinath2003/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)


,reviews_list,cuisines
11891,rated 20 ratedn menu differentthey dont hav pr...,"North Indian, Chinese"
5883,rated 40 ratedn 2012 first tried place till da...,"Street Food, Mithai"
8872,rated 50 ratedn another added feather indirana...,North Indian
8914,rated 50 ratedn mood spicy food wife suggested...,"Andhra, Biryani, North Indian"
11591,rated 50 ratedn awesome place come many times ...,"Chinese, North Indian, Continental"


In [53]:
zomato.head()

,name,cost,rate,reviews_list,cuisines,Mean Rating
0,Jalsa,800.0,4.1,rated 40 ratedn beautiful place dine inthe int...,"North Indian, Mughlai, Chinese",4.00
1,Spice Elephant,800.0,4.1,rated 40 ratedn dinner family turned good choo...,"Chinese, North Indian, Thai",3.97
2,San Churro Cafe,800.0,3.8,rated 30 ratedn ambience good enough pocket fr...,"Cafe, Mexican, Italian",3.58
3,Addhuri Udupi Bhojana,300.0,3.7,rated 40 ratedn great food proper karnataka st...,"South Indian, North Indian",3.45
4,Grand Village,600.0,3.8,rated 40 ratedn good restaurant neighbourhood ...,"North Indian, Rajasthani",3.58


In [54]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
    

import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Assuming df_percent is already defined
df_percent.set_index('name', inplace=True)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

# Compute cosine similarities
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


KeyboardInterrupt: 

In [ ]:
df_percent.head()

In [ ]:
df_percent['reviews_list'][0]

In [ ]:
index_value = "Bheema'S"  # replace with the actual index value you're checking
if index_value in df_percent.index:
    print(f"Index '{index_value}' is present in the DataFrame.")
else:
    print(f"Index '{index_value}' is not present in the DataFrame.")


In [ ]:
# Ensure that 'indices' is a mapping from restaurant names to their integer index
indices = pd.Series(df_percent.index, index=range(len(df_percent)))

def recommend(name, cosine_similarities=cosine_similarities):
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Check if the restaurant is in the DataFrame
    if name in df_percent.index:
        # Find the integer index of the restaurant
        idx = indices[df_percent.index == name].index[0]
    else:
        print(f"Restaurant '{name}' not found in the DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame if the restaurant is not found
    
    # Find the restaurants with a similar cosine similarity value and order them from highest to lowest
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine similarity value
    top30_indexes = list(score_series.iloc[1:31].index)  # Start from 1 to avoid recommending the same restaurant
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new DataFrame to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        temp_df = df_percent[['cuisines', 'Mean Rating', 'cost']][df_percent.index == each]
        if not temp_df.empty:  # Ensure the sampled DataFrame is not empty
            df_new = pd.concat([df_new, temp_df.sample()])
    
    # Drop duplicates and sort only the top 10 by the highest Mean Rating
    df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep='first')
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new


In [ ]:
recommend("Bheema'S") 

In [ ]:
'Jalsa' in restaurant_names

In [ ]:
restaurant_names